In [1]:
import json
import pandas as pd
from tqdm import tqdm
import sys

def read_data(file_name):
    items = []
    for i in open(file_name,'r').readlines():
        items.append(json.loads(i))
    return pd.DataFrame(items)
def save_data(df, o_name, suffix = 'json'):
    df = df.astype(object)
    with open(f"{o_name}.{suffix}",'w+') as t:
        for i in tqdm(range(len(df))):
            item = df.iloc[i,:].to_dict()
            t.write(json.dumps(item)+'\n')
def save_dict(d, o_name):
    with open(f"{o_name}.json",'w+') as o:
        o.write(json.dumps(d))

In [2]:
data_ccn = read_data('./CCN_filtered.json')
data_cp = read_data('./CommitPack_OldNew_Java.json')
data_cp_diffusion = read_data('./CommitPack_OldNew_Java_diffusion_num3_length_constrain.json')

In [3]:
data_ccn.head(2)

,msg,old,new,lang,proj
0,fix panic TestNonSessionPanicsAreNotCaught.,func TestNonSessionPanicsAreNotCaught(t *testi...,func TestNonSessionPanicsAreNotCaught(t *testi...,go,cockroachdb__copyist
1,offscreen render off,"function onRender(api, ctx, {bitmap, images, t...","function onRender(api, ctx, {bitmap, images, t...",javascript,d07riv__diabloweb


In [4]:
sum([1 if i == '' else 0 for i in  data_ccn.old.tolist()]), sum([1 if i == '' else 0 for i in  data_ccn.new.tolist()])

(67, 56)

In [5]:
# length bigger than 0
data_ccn = data_ccn.loc[(data_ccn.old != '') & (data_ccn.new != '') & (data_ccn.old != data_ccn.new )]

In [6]:
data_cp.head(2)

,commit,repos,msg,old,new,selected
0,dfe15087d9fe3bd8d8ba98c80bf2fc08d26f04f1,"rodsol/relex-temp,AmeBel/relex,opencog/relex,a...",print multiple parses\n,\t\t\t\tins = out_sock.getInputStream();\n\t\t...,\t\t\t\tins = out_sock.getInputStream();\n\t\t...,1
1,411840b9b28ac429ef463fad7e7cf69f4424272f,"kasemir/org.csstudio.display.builder,ESSICS/or...",Added support for SVG D&D.\n,private static Stroke OUTLINE_STROKE = new...,private static Stroke OUTLINE_STROKE = new...,1


In [7]:
sum([1 if i == '' else 0 for i in  data_cp.old.tolist()]), sum([1 if i == '' else 0 for i in  data_cp.new.tolist()])

(36396, 6598)

In [8]:
data_cp = data_cp.loc[(data_cp.old != '') & (data_cp.new != '') & (data_cp.old != data_cp.new )]

In [10]:
data_cp_diffusion.head(2)

,commit,repos,msg,old,new,selected
0,dfe15087d9fe3bd8d8ba98c80bf2fc08d26f04f1,"rodsol/relex-temp,AmeBel/relex,opencog/relex,a...",print multiple parses\n,\n\t\t\t\tins = out_sock.getInputStream();\n\t...,\t\t\t\tins = out_sock.getInputStream();\n\t\t...,1
1,dfe15087d9fe3bd8d8ba98c80bf2fc08d26f04f1,"rodsol/relex-temp,AmeBel/relex,opencog/relex,a...",print multiple parses\n,\t\t\t\tins = out_sock.getInputStream();\n\t\t...,\t\t\t\tins = out_sock.getInputStream();\n\t\t...,1


In [11]:
sum([1 if i == '' else 0 for i in  data_cp_diffusion.old.tolist()]), sum([1 if i == '' else 0 for i in  data_cp_diffusion.new.tolist()])

(0, 0)

In [12]:
data_cp_diffusion = data_cp_diffusion.loc[(data_cp_diffusion.old != '') & (data_cp_diffusion.new != '') & (data_cp_diffusion.old != data_cp_diffusion.new )]

In [13]:
df = pd.DataFrame({'proj': data_ccn.proj.tolist()+data_cp.repos.tolist()+data_cp_diffusion.repos.tolist(),
                    'nl': data_ccn.msg.tolist()+data_cp.msg.tolist()+data_cp_diffusion.msg.tolist(),
                    'old': data_ccn.old.tolist()+data_cp.old.tolist()+data_cp_diffusion.old.tolist(),
                    'new': data_ccn.new.tolist()+data_cp.new.tolist()+data_cp_diffusion.new.tolist(),
                    'lang':data_ccn.lang.tolist()+['java']*len(data_cp)+['java']*len(data_cp_diffusion),
                    'type': ['ccn']*len(data_ccn)+['commitpack']*len(data_cp)+['commitpack_diffusion']*len(data_cp_diffusion)})

In [14]:
df.reset_index(drop=True, inplace=True)
df.head(2)

,proj,nl,old,new,lang,type
0,cockroachdb__copyist,fix panic TestNonSessionPanicsAreNotCaught.,func TestNonSessionPanicsAreNotCaught(t *testi...,func TestNonSessionPanicsAreNotCaught(t *testi...,go,ccn
1,d07riv__diabloweb,offscreen render off,"function onRender(api, ctx, {bitmap, images, t...","function onRender(api, ctx, {bitmap, images, t...",javascript,ccn


In [15]:
len(df), len(data_ccn), len(data_cp), len(data_cp_diffusion)

(8898193, 1543017, 1633355, 5721821)

In [16]:
sum([1 if i == '' else 0 for i in  df.old.tolist()]), sum([1 if i == '' else 0 for i in  df.new.tolist()])

(0, 0)

In [17]:
data = df.sample(frac=1, random_state=42)

In [18]:
data.reset_index(drop=True, inplace=True)
data.head(4)

,proj,nl,old,new,lang,type
0,"jooby-project/jooby,jooby-project/jooby,jooby-...",add foreach test to rx2\n,Maybe.empty()\n .subscr...,Maybe.empty()\n .subscr...,java,commitpack_diffusion
1,"mariosotil/couchbase-lite-java-core,deleet/cou...",Fix for testReplicatorErrorStatus() that was f...,*/\n @InterfaceAudience.Public\n pu...,*/\n @InterfaceAudience.Public\n pu...,java,commitpack_diffusion
2,"karlthepagan/levelup-java-examples,MartinMSPed...",updated example per @TatuSaloranta feedback\n,"\t\t\t\tnew TypeReference<HashMap<String, Obje...","\t\t\t\tnew TypeReference<HashMap<String, Obje...",java,commitpack
3,rubygems__rubygems,Move TODO comment to an information comment,class Gem::DependencyInstaller\n if conside...,class Gem::DependencyInstaller\n if conside...,ruby,ccn


In [19]:
split_num = 30
for i in range(split_num):
    start = i*int(len(data)/split_num)
    end = (i+1)*int(len(data)/split_num)
    if i == split_num-1:
        end = len(data)
    save_data(data.iloc[start:end,:], f'./dataset_V7/train_{i}', suffix='jsonl')

100%|██████████| 296619/296619 [00:31<00:00, 9478.03it/s]
